# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
import json
import requests
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db =  mongo["uk_food"]

In [4]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [5]:
# assign the collection to a variable
establishments = db["establishments"]

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
print(f'The number of documents that hygiene is 20 is {establishments.count_documents(query)}')
# Display the first document in the results using pprint
establishments.find_one(query)

The number of documents that hygiene is 20 is 0


In [7]:
# Convert the result to a Pandas DataFrame
hygiene = establishments.find(query)
hygiene_df = pd.DataFrame(list(hygiene))
# Display the number of rows in the DataFrame
print(len(hygiene_df.index))
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'$and':[
    {"LocalAuthorityName":{"$regex":"London"}},
    {"RatingValue":{"$gte":4}}
]}

# Use count_documents to display the number of documents in the result
rated_4_count = establishments.count_documents(query)
print(rated_4_count)
# Display the first document in the results using pprint
establishments.find_one(query)

0


In [11]:
# Convert the result to a Pandas DataFrame
rated_4_count = establishments.find(query)
rated_4_count_df = pd.DataFrame(list(rated_4_count))
# Display the number of rows in the DataFrame
print(len(rated_4_count_df.index))
# Display the first 10 rows of the DataFrame
rated_4_count_df.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

max_lat = latitude+degree_search
min_lat = latitude-degree_search
max_long = longitude+degree_search
min_long = longitude-degree_search

query = {'$and':[{'geocode.latitude':{'$gte':min_lat},
                 'geocode.latitude':{'$lte':min_lat}}],
                 '$and':[{'geocode.longitude':{'$gte':min_long},
                 'geocode.longitude':{'$lte':max_long}}]
                ,'$and':[{'RatingValue':5}]}
sort = [("scores.Hygiene", 1)]
limit = 5
# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

establishments.count_documents(query)

[]


0

In [13]:
# Convert result to Pandas DataFrame
query = { "$and":[
        { "$and":[{ "geocode.latitude": {"$lte" : max_lat}}, {"geocode.latitude": {"$gte" : min_lat}}]},
        { "$and":[{ "geocode.longitude": {"$lte" : max_long}}, {"geocode.longitude": {"$gte" : min_long}}]},
        { "RatingValue": {"$eq" : "5"} }
    ]
}
establishments.count_documents(query)

0

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [15]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
matches_query = {"$match":{"scores.Hygiene":0}}
# 2. Groups the matches by Local Authority
group_query = {"$group":{'_id':'$LocalAuthorityName', 'count':{'$sum':1}}}
# 3. Sorts the matches from highest to lowest
sorts_matches = {'$sort': {'count':-1}}
pipeline = [matches_query, group_query, sorts_matches]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(len(results))
# Print the first 10 results
pprint(results[0:10])

0
[]


In [16]:
# Convert the result to a Pandas DataFrame
final_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(len(final_df))
# Display the first 10 rows of the DataFrame
final_df.head(10)

0


""
